<a href="https://colab.research.google.com/github/Rakshithabmsce/UserReady/blob/main/extracting_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy python-docx
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
from google.colab import files
uploaded = files.upload()

Saving 24158401-Rental-Agreement.png to 24158401-Rental-Agreement.png
Saving 95980236-Rental-Agreement.png to 95980236-Rental-Agreement.png
Saving 156155545-Rental-Agreement-Kns-Home.pdf.docx to 156155545-Rental-Agreement-Kns-Home.pdf.docx
Saving 228094620-Rental-Agreement.pdf.docx to 228094620-Rental-Agreement.pdf.docx


In [3]:
import spacy
from docx import Document
import pandas as pd

nlp = spacy.load("en_core_web_sm")

In [6]:
def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_fields(text):
    doc = nlp(text)
    fields = {
        "Agreement Value": "",
        "Agreement Start Date": "",
        "Agreement End Date": "",
        "Renewal Notice (Days)": "",
        "Party One": "",
        "Party Two": ""
    }

    for ent in doc.ents:
        if ent.label_ == "MONEY" and not fields["Agreement Value"]:
            fields["Agreement Value"] = ent.text

    parties = [ent.text for ent in doc.ents if ent.label_ in ["ORG", "PERSON"]]
    if len(parties) >= 2:
        fields["Party One"], fields["Party Two"] = parties[0], parties[1]

    dates = [ent.text for ent in doc.ents if ent.label_ == "DATE"]
    if len(dates) >= 2:
        fields["Agreement Start Date"], fields["Agreement End Date"] = dates[0], dates[1]

    for token in doc:
        if token.like_num and "day" in token.sent.text.lower():
            fields["Renewal Notice (Days)"] = token.text
            break

    return fields

In [9]:
!apt install tesseract-ocr
!pip install pytesseract pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [10]:
import os
from PIL import Image
import pytesseract

# Function to extract text from .png using OCR
def extract_text_from_image(image_path):
    return pytesseract.image_to_string(Image.open(image_path))

results = []

for filename in uploaded.keys():
    if filename.lower().endswith(".docx"):
        text = extract_text_from_docx(filename)
    elif filename.lower().endswith(".png"):
        text = extract_text_from_image(filename)
    else:
        print(f"Unsupported file: {filename}")
        continue

    data = extract_fields(text)
    data["Filename"] = filename
    results.append(data)


In [11]:
df = pd.DataFrame(results)
df = df[["Filename", "Agreement Value", "Agreement Start Date", "Agreement End Date",
         "Renewal Notice (Days)", "Party One", "Party Two"]]
df.to_csv("test_predictions.csv", index=False)

In [12]:
files.download("test_predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>